In [ ]:
from sys import path
from os.path import realpath
path.append(realpath("../")) # Makes it works everywhere
from PIL import Image
import numpy as np

In [75]:
path = 'imgs/linea.png'
image = Image.open(path).convert('L')
image = np.asarray(image)

encoded_b2 = ""
for i in image:
    for j, el in enumerate(i):
        encoded_b2 += format(el, '08b')

encoded_b2 = '\n'.join(encoded_b2[i:i+8] for i in range(0, len(encoded_b2), 8))

In [76]:
with open('VHDL/images_in_fpga/src/linea.mif', 'w')  as outfile:   
    outfile.write(encoded_b2)

Falta la vuelta (txt2png)